<a href="https://colab.research.google.com/github\nicoG2023/Data_Science_Final_Project/blob/Predata/src/data-science-project/data_science_project/EDA_Rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA "Grupo Bimbo Inventory Demand"

### Introduction

Grupo Bimbo, a leading multinational bakery company, faces a unique challenge in managing the inventory of its products. With a typical shelf life of just one week, the accuracy of daily inventory calculations is paramount. Currently, these calculations are performed by direct delivery sales employees who rely on their personal experiences to predict the forces of supply, demand, and consumer behavior at each store. The margin for error in this process is minimal. Underestimating demand results in empty shelves and lost sales, while overestimating demand leads to excess product returns and increased expenses.

Grupo Bimbo aims to create a predictive model that can accurately forecast inventory needs based on historical data, thereby optimizing the supply chain and improving efficiency.


## Libraries

In [1]:
# importing the basic libraries
!pip install ydata_profiling
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import path
import os
import plotly.express as px
import gc

^C


ModuleNotFoundError: No module named 'ydata_profiling'

## Charge data

In [2]:
train_df = pd.read_csv("C:/Users/ADMIN/OneDrive - Universidad Distrital Francisco José de Caldas/Data_Science_Project/grupo-bimbo-inventory-demand/train.csv.zip", compression='zip')
client_df = pd.read_csv("C:/Users/ADMIN/OneDrive - Universidad Distrital Francisco José de Caldas/Data_Science_Project/grupo-bimbo-inventory-demand/client_tabla.csv.zip", compression='zip')
products_df = pd.read_csv("C:/Users/ADMIN/OneDrive - Universidad Distrital Francisco José de Caldas/Data_Science_Project/grupo-bimbo-inventory-demand/producto_tabla.csv.zip", compression='zip')
test_df = pd.read_csv("C:/Users/ADMIN/OneDrive - Universidad Distrital Francisco José de Caldas/Data_Science_Project/grupo-bimbo-inventory-demand/test.csv.zip", compression='zip')
town_state_df = pd.read_csv("C:/Users/ADMIN/OneDrive - Universidad Distrital Francisco José de Caldas/Data_Science_Project/grupo-bimbo-inventory-demand/town_state.csv.zip", compression='zip')

Training Size : (1126694,20)
Test Size : (630452,19)


In [1]:
# Inspect the data, with funcion

def inspect_columns(df, df_name):
    """A helper function that does a better job than df.info() and df.describe()"""
    
    total_rows = len(df)
    result = pd.DataFrame({
        'total_rows': [total_rows] * df.shape[1],
        'rows_with_missing_values': df.isnull().sum(),
        'unique': df.nunique() == total_rows,
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum() / total_rows) * 100, 2),
        '1st_row': df.iloc[0],
        'random_row': df.iloc[np.random.randint(low=0, high=total_rows)],
        'last_row': df.iloc[-1],
        'dtype': df.dtypes,
    })
    
    # Print the name of the dataframe
    print(f"\n{'='*10} {df_name} {'='*10}\n")
    
    # Print the head of the dataframe
    print("First few rows of the dataframe:\n")
    display(df.head())
    
    # Print the resulting statistics
    print("Detailed statistics:\n")
    display(result)

## Understand the data

We performed a preliminary exploration to understand the structure of the data.

In [2]:
inspect_columns(train_df,"Training Dataframe")
inspect_columns(client_df,"Client Dataframe")
inspect_columns(products_df,"Products Dataframe")
inspect_columns(test_df,"Test Dataframe")
inspect_columns(town_state_df,"Town and State Dataframe")

NameError: name 'train_df' is not defined

In [ ]:
# inspect the data with ydata profile

profile_obj = ProfileReport(train_df, title='Bimbo Inventory Demand Data Profiling Report', explorative=True)
profile_obj.to_file('/content/drive/MyDrive/train_final.html')
profile_obj

### Exploratory data analysis


In [3]:
# transform by merge with the data
    
train_df = train_df.merge(client_df,on = "Cliente_ID",how="left")
gc.collect()
train_df = train_df.merge(products_df,on = "Producto_ID",how="left")
gc.collect()
train_df = train_df.merge(town_state_df,on = "Agencia_ID",how="left")
gc.collect()

train_df.head() 

In [ ]:
# translate the language used for the data

train_df = train_df.rename({'Semana':'Week', 'Agencia_ID':'Sales_Depot_ID', 'Canal_ID':'Sales_Channel_ID', 'Ruta_SAK':'Route_ID', 'Cliente_ID':'Client_ID',
       'Producto_ID':'Product_ID', 'Venta_uni_hoy':'Sales_unit_this_week ', 'Venta_hoy':'Sales_this_week', 'Dev_uni_proxima':'Returns_unit_next_week',
       'Dev_proxima':'Returns_next_week', 'Demanda_uni_equil':'Adjusted_Demand', 'NombreCliente':'Client_name', 'NombreProducto':'Product_Name',
       'Town':'Town', 'State':'State'},axis=1)

train_df.head() 